In [ ]:
import numpy as np
import random
import os
import math

from glob import glob
import pandas as pd
import cv2
from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

import torchvision.models as models
from torchvision import transforms

: 

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

In [ ]:
def get_train_data(data_dir):
    img_path_list = []
    label_list = []
    for case_name in os.listdir(data_dir):
        current_path = os.path.join(data_dir, case_name)
        if os.path.isdir(current_path):
            # get image path
            img_path_list.extend(glob(os.path.join(current_path, 'image', '*.jpg')))
            img_path_list.extend(glob(os.path.join(current_path, 'image', '*.png')))
            
            # get label
            label_df = pd.read_csv(current_path+'/label.csv')
            label_list.extend(label_df['leaf_weight'])
                
    return img_path_list, label_list

def get_test_data(data_dir):
    # get image path
    img_path_list = glob(os.path.join(data_dir, 'image', '*.jpg'))
    img_path_list.extend(glob(os.path.join(data_dir, 'image', '*.png')))
    img_path_list.sort(key=lambda x:int(x.split('\\')[-1].split('.')[0]))
    return img_path_list

In [ ]:
trainpath = glob.glob(os.path.join('.', 'dataset_128', 'train', 'CASE*', 'image', '*'))
traintensor = torch.Tensor(size=[0,3,128,128])
for index, i in enumerate(trainpath):
    img = Image.open(i)
    transform = transforms.ToTensor()
    temp = transform(img)
    temp = temp.unsqueeze(dim=0)
    # traintensor.cat(temp)
    traintensor = torch.cat([traintensor, temp],dim = 0)
    if index%100==0:
        print(1)
traintensor.shape


In [ ]:
from augmentation import *
import random
import time
def augment(traintensor):
    augmenttraintensor = torch.Tensor(size = [0,3,128,128])
    for index, image in enumerate(traintensor):
        start = time.time()
        rand = random.random()
        #image = image.unsqueeze(dim = 0)
        #print(image.shape)
        if rand<0.33:
            temp_1 = iso_scaling(image)
            temp_2 = gau_blur(image)
            temp_1 = temp_1.unsqueeze(dim = 0)
            temp_2 = temp_2.unsqueeze(dim = 0)
        elif rand<0.67 and rand>=0.33:
            temp_1 = rand_erasing(image)
            temp_2 = gau_noise(image)
            temp_1 = temp_1.unsqueeze(dim = 0)
            temp_2 = temp_2.unsqueeze(dim = 0)
        else:
            temp_1 = salt_pepper(image)
            temp_2 = hue_rotation(image)
            temp_1 = temp_1.unsqueeze(dim = 0)
            temp_2 = temp_2.unsqueeze(dim = 0)
        image = image.unsqueeze(dim = 0)
        augmenttraintensor = torch.cat([augmenttraintensor, temp_1],dim=0)
        augmenttraintensor = torch.cat([augmenttraintensor, temp_2],dim=0)
        augmenttraintensor = torch.cat([augmenttraintensor, image],dim=0)
        if index%100==0:
            print("------", index)
            print(time.time() - start)
    return augmenttraintensor

In [ ]:
traintensor1 = traintensor[:400]
traintensor2 = traintensor[400:800]
traintensor3 = traintensor[800:1200]
traintensor4 = traintensor[1200:]
traintensor_all = torch.Tensor(size = [0,3,128,128])
traintensor_all = torch.cat([traintensor_all, augment(traintensor1)], dim = 0)
print("&&&&&")
traintensor_all = torch.cat([traintensor_all, augment(traintensor2)], dim = 0)
print("&&&&&")
traintensor_all = torch.cat([traintensor_all, augment(traintensor3)], dim = 0)
print("&&&&&")
traintensor_all = torch.cat([traintensor_all, augment(traintensor4)], dim = 0)
print(traintensor_all.shape)

In [ ]:
all_img_path, all_label = get_train_data('./dataset/train')
test_img_path = get_test_data('./dataset/test')

In [ ]:
# Train : Validation = 0.8 : 0.2 Split
train_len = int(len(all_img_path)*0.8)

train_img_path = all_img_path[:train_len]
train_label = all_label[:train_len]

vali_img_path = all_img_path[train_len:]
vali_label = all_label[train_len:]

In [ ]:
len(train_label)+len(vali_label)
train_all_label_1 = train_label+vali_label
train_all_label = []
for i in train_all_label_1:
    train_all_label.append(i)
    train_all_label.append(i)
    train_all_label.append(i)
len(train_all_label)
    

In [ ]:
trainimage_tensor = traintensor_all[:1250]
trainlabel = train_all_label[:1250]
validimage_tensor = traintensor_all[1250:]
validlabel = train_all_label[1250:]